In [114]:
import ast
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_recommenders as tfrs
from typing import Dict, Text


In [115]:
raw_user_data = pd.read_json('../results/user_dataset.json')
raw_food_data = pd.read_json('../results/output.json')
listUserPreferences = pd.DataFrame(
    [data for data in raw_user_data.listUserPreferences])
listFoodPreferences = pd.DataFrame(
    [data for data in raw_food_data.food])
temp = pd.DataFrame(
    [data for data in listFoodPreferences.servings])
listFoodServings = pd.DataFrame(
    [item for sublist in temp.serving for item in sublist])
listFoodPreferences = listFoodPreferences.drop(columns=['servings'])
raw_listFood = listFoodPreferences.merge(
    listFoodServings, how='left', left_index=True, right_index=True)
listFood = raw_listFood.fillna(0)


In [116]:
listUserPreferences


,gender,weight,id_user,activityLevel,goals,height,disease,favoriteFood,birthdate
0,Male,50,3222915,1.9,0,180,[],"[Chicken Cordon Bleu, Chocolate Mousse Cake, S...",20-10-1990
1,Male,46,9930952,1.725,0,151,[obesity],"[Samosa, Pizza Dough, Baguette, Chocolate Mous...",10-09-1976
2,Male,60,3717984,1.725,0,164,"[heart, hypertension]","[Bubur Ayam, Mashed Potato, Puff Pastry, Chick...",01-05-2019
3,Male,41,6726690,1.55,0,156,"[diabetes, heart, obesity]","[Fish Cake, Red Velvet, Black Forest, Guacamol...",02-06-1979
4,Female,66,6452326,1.2,-500,151,[],"[Puff Pastry, Mashed Potato, Bubur Ayam, Okono...",22-03-1988
...,...,...,...,...,...,...,...,...,...
260,Male,65,7559844,1.55,500,163,[],"[Chicken Cordon Bleu, Pad Thai, Churros, Black...",26-01-1975
261,Male,84,8260717,1.9,500,196,[heart],"[Bibimbap, Puff Pastry, Thai Tea, Churros, Cin...",14-05-1950
262,Female,47,4895731,1.55,-500,180,"[diabetes, heart]","[Chocolate Lava Cake, Meringue Cookies, Takoya...",21-07-2016
263,Male,63,7540747,1.55,500,183,"[diabetes, hypertension, obesity]","[Thai Tea, Chocolate Mousse Cake, Pad Thai, Me...",22-10-1952


In [117]:
listFood


,food_id,food_name,food_type,food_url,brand_name,calcium,calories,carbohydrate,cholesterol,fat,...,serving_description,serving_id,serving_url,sodium,sugar,trans_fat,vitamin_a,vitamin_c,added_sugars,vitamin_d
0,2655309,Kebab,[],https://www.fatsecret.com/calories-nutrition/g...,0,176,620,77.10,53,16.63,...,1 kebab,2590791,https://www.fatsecret.com/calories-nutrition/g...,1029,6.91,0,37,22.2,0,0
1,58704864,Gyoza,[],https://www.fatsecret.com/calories-nutrition/h...,Hungry Planet,176,620,77.10,53,16.63,...,1 serving (390 g),2590792,https://www.fatsecret.com/calories-nutrition/g...,1029,6.91,0,37,22.2,0,0
2,68848666,Gyoza,[],https://www.fatsecret.com/calories-nutrition/y...,Yoshinoya,45,159,19.77,13,4.27,...,100 g,2590793,https://www.fatsecret.com/calories-nutrition/g...,264,1.77,0,9,5.7,0,0
3,515347,Gyoza,[],https://www.fatsecret.com/calories-nutrition/t...,Tokyo Joe's,13,45,5.60,4,1.21,...,1 oz,2590794,https://www.fatsecret.com/calories-nutrition/g...,75,0.50,0,3,1.6,0,0
4,1684073,Chicken Katsu,[],https://www.fatsecret.com/calories-nutrition/l...,L&L Hawaiian Barbecue,30,120,17.00,0,2.50,...,4 gyoza,48898797,https://www.fatsecret.com/calories-nutrition/h...,280,1.00,0,0,0,1.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,447317,Guacamole,[],https://www.fatsecret.com/calories-nutrition/c...,Chevy's Fresh Mex,0,330,41.00,0,15.00,...,1 serving,56535626,https://www.fatsecret.com/calories-nutrition/r...,390,6.00,0.500,0,0,0,0
261,2125933,Guacamole,[],https://www.fatsecret.com/calories-nutrition/c...,Costa Vida,0,100,12.00,0,4.50,...,1 croissant,44521664,https://www.fatsecret.com/calories-nutrition/p...,210,3.00,0,0,0,0,0
262,1995633,Guacamole,[],https://www.fatsecret.com/calories-nutrition/o...,Ortega,0,264,28.90,0,13.80,...,1 serving,56209720,https://www.fatsecret.com/calories-nutrition/t...,246,3.90,0.100,0,0,0,0
263,99383,Guacamole,[],https://www.fatsecret.com/calories-nutrition/c...,Chili's,0,250,25.00,35,14.00,...,1 serving,4438697,https://www.fatsecret.com/calories-nutrition/s...,230,3.00,0,0,0,0,0


In [118]:
# Convert the list columns to strings
listFood['food_name'] = listFood['food_name'].astype(str)
# listUserPreferences['favoriteFood'] = listUserPreferences['favoriteFood'].astype(str).apply(eval).apply(lambda x: ', '.join(x))
listUserPreferences = listUserPreferences.explode('favoriteFood')
listFood['food_name']


0              Kebab
1              Gyoza
2              Gyoza
3              Gyoza
4      Chicken Katsu
           ...      
260        Guacamole
261        Guacamole
262        Guacamole
263        Guacamole
264        Guacamole
Name: food_name, Length: 265, dtype: object

In [119]:

listUserPreferences['favoriteFood']


0        Chicken Cordon Bleu
0      Chocolate Mousse Cake
0                     Samosa
0               Cream Cheese
0                Carrot Cake
               ...          
264          Salmon Teriyaki
264               Burger Bun
264                 Pad Thai
264              Onion Rings
264                 Takoyaki
Name: favoriteFood, Length: 1981, dtype: object

In [120]:
# Merge the DataFrames based on the common attribute, such as food name or ID
raw_merged_df = pd.merge(listFood, listUserPreferences,
                         left_on='food_name', right_on='favoriteFood', how='inner')

raw_merged_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 11193 entries, 0 to 11192
Data columns (total 40 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   food_id                  11193 non-null  object
 1   food_name                11193 non-null  object
 2   food_type                11193 non-null  object
 3   food_url                 11193 non-null  object
 4   brand_name               11193 non-null  object
 5   calcium                  11193 non-null  object
 6   calories                 11193 non-null  object
 7   carbohydrate             11193 non-null  object
 8   cholesterol              11193 non-null  object
 9   fat                      11193 non-null  object
 10  fiber                    11193 non-null  object
 11  iron                     11193 non-null  object
 12  measurement_description  11193 non-null  object
 13  metric_serving_amount    11193 non-null  object
 14  metric_serving_unit      11193 non-nul

In [121]:
raw_merged_df

,food_id,food_name,food_type,food_url,brand_name,calcium,calories,carbohydrate,cholesterol,fat,...,vitamin_d,gender,weight,id_user,activityLevel,goals,height,disease,favoriteFood,birthdate
0,2655309,Kebab,[],https://www.fatsecret.com/calories-nutrition/g...,0,176,620,77.10,53,16.63,...,0,Male,43,3847302,1.55,0,196,"[obesity, diabetes, heart, hypertension]",Kebab,09-01-2019
1,2655309,Kebab,[],https://www.fatsecret.com/calories-nutrition/g...,0,176,620,77.10,53,16.63,...,0,Male,54,2935940,1.55,500,189,"[heart, diabetes, obesity, hypertension]",Kebab,26-08-1967
2,2655309,Kebab,[],https://www.fatsecret.com/calories-nutrition/g...,0,176,620,77.10,53,16.63,...,0,Male,40,5259626,1.2,0,199,"[heart, hypertension, obesity, diabetes]",Kebab,20-01-1972
3,2655309,Kebab,[],https://www.fatsecret.com/calories-nutrition/g...,0,176,620,77.10,53,16.63,...,0,Male,82,2097001,1.2,500,181,"[hypertension, diabetes, heart, obesity]",Kebab,20-03-1978
4,2655309,Kebab,[],https://www.fatsecret.com/calories-nutrition/g...,0,176,620,77.10,53,16.63,...,0,Male,84,7545290,1.375,-500,153,[diabetes],Kebab,21-04-2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11188,69852,Guacamole,[],https://www.fatsecret.com/calories-nutrition/f...,Freebirds,0,340,39.00,60,16.00,...,0,Female,92,8630293,1.2,0,159,[hypertension],Guacamole,28-12-2020
11189,69852,Guacamole,[],https://www.fatsecret.com/calories-nutrition/f...,Freebirds,0,340,39.00,60,16.00,...,0,Male,70,3959643,1.375,500,163,"[diabetes, obesity, heart]",Guacamole,27-02-2005
11190,69852,Guacamole,[],https://www.fatsecret.com/calories-nutrition/f...,Freebirds,0,340,39.00,60,16.00,...,0,Female,70,9023340,1.2,-500,191,"[hypertension, diabetes, heart, obesity]",Guacamole,24-11-1965
11191,69852,Guacamole,[],https://www.fatsecret.com/calories-nutrition/f...,Freebirds,0,340,39.00,60,16.00,...,0,Female,98,1433326,1.55,500,164,[obesity],Guacamole,25-06-1991


In [122]:
merged_df = pd.DataFrame()
merged_df['food_id'] = raw_merged_df['food_id'].astype(str)
merged_df['food_name'] = raw_merged_df['food_name'].astype(str)
merged_df['brand_name'] = raw_merged_df['brand_name'].astype(str)
merged_df['id_user'] = raw_merged_df['id_user'].astype(str)
merged_df['gender'] = raw_merged_df['gender'].astype(str)
merged_df['disease'] = raw_merged_df['disease'].apply(
    lambda x: ', '.join(x) if isinstance(x, list) else str(x))
merged_df['favoriteFood'] = raw_merged_df['favoriteFood'].astype(str)

merged_df


,food_id,food_name,brand_name,id_user,gender,disease,favoriteFood
0,2655309,Kebab,0,3847302,Male,"obesity, diabetes, heart, hypertension",Kebab
1,2655309,Kebab,0,2935940,Male,"heart, diabetes, obesity, hypertension",Kebab
2,2655309,Kebab,0,5259626,Male,"heart, hypertension, obesity, diabetes",Kebab
3,2655309,Kebab,0,2097001,Male,"hypertension, diabetes, heart, obesity",Kebab
4,2655309,Kebab,0,7545290,Male,diabetes,Kebab
...,...,...,...,...,...,...,...
11188,69852,Guacamole,Freebirds,8630293,Female,hypertension,Guacamole
11189,69852,Guacamole,Freebirds,3959643,Male,"diabetes, obesity, heart",Guacamole
11190,69852,Guacamole,Freebirds,9023340,Female,"hypertension, diabetes, heart, obesity",Guacamole
11191,69852,Guacamole,Freebirds,1433326,Female,obesity,Guacamole


In [123]:
user = merged_df[['id_user', 'gender', 'disease', 'favoriteFood']]
user


,id_user,gender,disease,favoriteFood
0,3847302,Male,"obesity, diabetes, heart, hypertension",Kebab
1,2935940,Male,"heart, diabetes, obesity, hypertension",Kebab
2,5259626,Male,"heart, hypertension, obesity, diabetes",Kebab
3,2097001,Male,"hypertension, diabetes, heart, obesity",Kebab
4,7545290,Male,diabetes,Kebab
...,...,...,...,...
11188,8630293,Female,hypertension,Guacamole
11189,3959643,Male,"diabetes, obesity, heart",Guacamole
11190,9023340,Female,"hypertension, diabetes, heart, obesity",Guacamole
11191,1433326,Female,obesity,Guacamole


In [124]:
food = merged_df[['food_id', 'food_name', 'brand_name']]
food

,food_id,food_name,brand_name
0,2655309,Kebab,0
1,2655309,Kebab,0
2,2655309,Kebab,0
3,2655309,Kebab,0
4,2655309,Kebab,0
...,...,...,...
11188,69852,Guacamole,Freebirds
11189,69852,Guacamole,Freebirds
11190,69852,Guacamole,Freebirds
11191,69852,Guacamole,Freebirds


In [125]:
user = tf.data.Dataset.from_tensor_slices(dict(user))
food = tf.data.Dataset.from_tensor_slices(dict(food))


In [126]:
user = user.map(lambda x: {
    "id_user": x["id_user"],
    "gender": x["gender"],
    "disease": x["disease"],
    "favoriteFood": x["favoriteFood"]
})

food = food.map(lambda x: {
    "food_id": x["food_id"],
    "food_name": x["food_name"],
    "brand_name": x["brand_name"]
})


In [127]:
user_array = []
food_array = []
for data in user:
    user_array.append(data['id_user'].numpy().decode())
for data in food:
    food_array.append(data['food_id'].numpy().decode())


#skills_array = tf.squeeze(skills).numpy().astype(str).tolist()
#jobs_array = tf.squeeze(jobs).numpy().astype(str).tolist()

user_ids_vocabulary = tf.keras.layers.StringLookup()
user_ids_vocabulary.adapt(user_array)

food_titles_vocabulary = tf.keras.layers.StringLookup()
food_titles_vocabulary.adapt(food_array)


In [128]:
class Model(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
          self,
          user_model: tf.keras.Model,
          food_model: tf.keras.Model,
          task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.food_model = food_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["id_user"])
    food_embeddings = self.food_model(features["food_name"])

    return self.task(user_embeddings, food_embeddings)


In [137]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    tf.keras.Input(shape=[], dtype=tf.string, name='id_user'),
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
food_model = tf.keras.Sequential([
    tf.keras.Input(shape=[], dtype=tf.string, name='food_id'),
    food_titles_vocabulary,
    tf.keras.layers.Embedding(food_titles_vocabulary.vocabulary_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        food.batch(128).map(food_model)
    )
)


In [138]:
# Create a retrieval model.
model = Model(user_model, food_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(food.batch(4096), epochs=3)


Epoch 1/3


KeyError: in user code:

    File "c:\Users\Ananda\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Ananda\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Ananda\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Ananda\anaconda3\lib\site-packages\tensorflow_recommenders\models\base.py", line 68, in train_step
        loss = self.compute_loss(inputs, training=True)
    File "C:\Users\Ananda\AppData\Local\Temp\ipykernel_7832\254163675.py", line 22, in compute_loss
        user_embeddings = self.user_model(features["id_user"])

    KeyError: 'id_user'


In [95]:
feature_columns = [
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "food_id", merged_df["food_id"].unique()),
        dimension=32
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "food_name", merged_df["food_name"].unique()),
        dimension=32
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "brand_name", merged_df["brand_name"].unique()),
        dimension=16
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "id_user", merged_df["id_user"].unique()),
        dimension=16
    ),
    tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "gender", merged_df["gender"].unique()),
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "favoriteFood", merged_df["favoriteFood"].unique()),
        dimension=8
    ),
]


In [96]:
feature_columns

[EmbeddingColumn(categorical_column=VocabularyListCategoricalColumn(key='food_id', vocabulary_list=('2655309', '58704864', '68848666', '515347', '1684073', '360970', '51780578', '35362184', '1751238', '59433475', '3451226', '1565707', '855710', '6926210', '50672307', '226469', '54876480', '1277715', '2277254', '6721055', '63385', '5405110', '57008676', '2464847', '195215', '1866283', '22836405', '36346', '94228', '48317129', '1905252', '15766420', '406466', '12173623', '3080414', '372433', '9238078', '7556379', '3310602', '152278', '6708', '110855', '3298228', '13582137', '4689586', '41361247', '30316181', '28343875', '19351319', '16694983', '46783291', '54274347', '12100664', '13385806', '6231843', '59526', '568852', '5569712', '1023391', '1824240', '745530', '4055482', '13753575', '71665', '2960716', '170061', '82162', '91509', '13418168', '69128', '1676706', '1155703', '3249398', '247366', '5777', '69083', '361022', '1460350', '4826631', '5873757', '65437539', '17775578', '14718941'

In [97]:
# Define a model for users and another for items (foods).
# Define the architecture of your user model
user_model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1)
])

# Define the architecture of your item model
item_model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1)
])


In [98]:
# Convert user_ids and food_ids to dictionaries.
user_ids = {"user_id": merged_df["id_user"].values}
food_ids = {"food_id": merged_df["food_id"].values}

In [99]:
# Create a retrieval task.
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=tf.data.Dataset.from_tensor_slices(food_ids)
        .batch(128)
        .map(item_model)
    )
)


ValueError: Exception encountered when calling layer "dense_features_13" "                 f"(type DenseFeatures).

Feature food_name is not in features dictionary.

Call arguments received by layer "dense_features_13" "                 f"(type DenseFeatures):
  • features={'food_id': 'tf.Tensor(shape=(None,), dtype=string)'}
  • cols_to_output_tensors=None
  • training=False

In [ ]:


# Combine them into a recommendation model.
model = tfrs.Model(user_model=user_model, item_model=item_model, task=task)

# Make a dataset of (user, item) pairs.
dataset = tf.data.Dataset.from_tensor_slices((user_ids, food_ids))
dataset = dataset.shuffle(len(merged_df)).batch(
    128).cache().prefetch(tf.data.experimental.AUTOTUNE)

# Fit the model.
model.fit(dataset, epochs=10)


NameError: name 'task' is not defined

In [ ]:
# TFRS imports.
import tensorflow_recommenders as tfrs

# Assume user_ids and food_ids are the relevant columns.
# Convert these to tensors.
user_ids = tf.constant(merged_df["id_user"])
food_ids = tf.constant(merged_df["food_id"])

# Create a retrieval task.
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=food_ids.batch(128).map(item_model)
    )
)

# Define a model for users and another for items (foods).
# define the architecture of your user model
user_model = tf.keras.Sequential(tf.keras.layers.DenseFeatures(feature_columns),
                                 tf.keras.layers.Dense(128, activation="relu"),
                                 tf.keras.layers.Dense(64, activation="relu"),
                                 tf.keras.layers.Dense(32, activation="relu"),
                                 tf.keras.layers.Dense(16, activation="relu"),
                                 tf.keras.layers.Dense(1))
# define the architecture of your item model
item_model = tf.keras.Sequential(tf.keras.layers.DenseFeatures(feature_columns),
                                 tf.keras.layers.Dense(128, activation="relu"),
                                 tf.keras.layers.Dense(64, activation="relu"),
                                 tf.keras.layers.Dense(32, activation="relu"),
                                 tf.keras.layers.Dense(16, activation="relu"),
                                 tf.keras.layers.Dense(1))

# Combine them into a recommendation model.
model = tfrs.Model(user_model=user_model, item_model=item_model, task=task)

# Train.
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

# Make a dataset of (user, item) pairs.
dataset = tf.data.Dataset.from_tensor_slices((user_ids, food_ids))
dataset = dataset.batch(128).cache().prefetch(tf.data.experimental.AUTOTUNE)

# Assume user_ids and food_ids are the relevant columns.
dataset = tf.data.Dataset.from_tensor_slices(
    (merged_df["user_id"].values, merged_df["food_id"].values))

# Shuffle and batch the dataset.
batch_size = 128
dataset = dataset.shuffle(len(merged_df)).batch(
    batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)

# Fit the model.
model.fit(dataset, epochs=10)


NameError: name 'item_model' is not defined

In [ ]:
import tensorflow as tf
import numpy as np

# Convert the Pandas DataFrame to a TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices(dict(merged_df))

# Shuffle and batch the dataset (adjust the batch_size as needed)
batch_size = 32
dataset = dataset.shuffle(len(merged_df)).batch(batch_size)

# Create an instance of the retrieval task
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=merged_df["food_id"].unique().tolist()
    )
)

# Create an instance of the TFRS model and compile it
model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1)
])

loss = tf.keras.losses.MeanSquaredError()

model.compile(optimizer=tf.keras.optimizers.Adagrad(
    learning_rate=0.1), loss=loss)

# Train the model with the TensorFlow Dataset
model.fit(dataset, epochs=10, verbose=1)


Epoch 1/10


ValueError: in user code:

    File "c:\Users\Ananda\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Ananda\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Ananda\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Ananda\anaconda3\lib\site-packages\keras\engine\training.py", line 995, in train_step
        self._validate_target_and_loss(y, loss)
    File "c:\Users\Ananda\anaconda3\lib\site-packages\keras\engine\training.py", line 949, in _validate_target_and_loss
        raise ValueError(

    ValueError: Target data is missing. Your model was compiled with loss=<keras.losses.MeanSquaredError object at 0x00000285853C6130>, and therefore expects target data to be provi

In [ ]:
import tensorflow as tf
import tensorflow_recommenders as tfrs

merged_df["disease"] = merged_df["disease"].apply(lambda x: ', '.join(x))

# Define the feature columns
feature_columns = [
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "food_id", merged_df["food_id"].unique()),
        dimension=32
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "food_name", merged_df["food_name"].unique()),
        dimension=32
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "food_type", merged_df["food_type"].unique()),
        dimension=8
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "brand_name", merged_df["brand_name"].unique()),
        dimension=16
    ),
    tf.feature_column.numeric_column("calcium"),
    tf.feature_column.numeric_column("calories"),
    tf.feature_column.numeric_column("carbohydrate"),
    tf.feature_column.numeric_column("cholesterol"),
    tf.feature_column.numeric_column("fat"),
    tf.feature_column.numeric_column("fiber"),
    tf.feature_column.numeric_column("iron"),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "measurement_description", merged_df["measurement_description"].unique()),
        dimension=8
    ),
    tf.feature_column.numeric_column("metric_serving_amount"),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "metric_serving_unit", merged_df["metric_serving_unit"].unique()),
        dimension=8
    ),
    tf.feature_column.numeric_column("monounsaturated_fat"),
    tf.feature_column.numeric_column("number_of_units"),
    tf.feature_column.numeric_column("polyunsaturated_fat"),
    tf.feature_column.numeric_column("potassium"),
    tf.feature_column.numeric_column("protein"),
    tf.feature_column.numeric_column("saturated_fat"),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "serving_description", merged_df["serving_description"].unique()),
        dimension=8
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "serving_id", merged_df["serving_id"].unique()),
        dimension=32
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "favoriteFood", merged_df["favoriteFood"].unique()),
        dimension=8
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "birthdate", merged_df["birthdate"].unique()),
        dimension=16
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "gender", merged_df["gender"].unique()),
        dimension=8
    ),
    tf.feature_column.numeric_column("weight"),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "goals", merged_df["goals"].unique()),
        dimension=8
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "disease", merged_df["disease"].unique().astype(str)),
        dimension=8
    )
]

# Create an instance of the TFRS model and compile it
model = tfrs.metrics.FactorizedTopK(
    model=tfrs.tasks.Retrieval(task))

# Train the model with the feature columns
model.fit(merged_df, epochs=10, verbose=1)


NameError: name 'task' is not defined

In [ ]:
# Convert columns to appropriate data types
merged_df['food_id'] = merged_df['food_id'].astype(str)
merged_df['food_name'] = merged_df['food_name'].astype(str)
merged_df['food_type'] = merged_df['food_type'].astype(str)
merged_df['food_url'] = merged_df['food_url'].astype(str)
merged_df['brand_name'] = merged_df['brand_name'].astype(str)
merged_df['calcium'] = merged_df['calcium'].astype(float)
merged_df['calories'] = merged_df['calories'].astype(float)
merged_df['carbohydrate'] = merged_df['carbohydrate'].astype(float)
merged_df['cholesterol'] = merged_df['cholesterol'].astype(float)
merged_df['fat'] = merged_df['fat'].astype(float)
merged_df['fiber'] = merged_df['fiber'].astype(float)
merged_df['iron'] = merged_df['iron'].astype(float)
merged_df['measurement_description'] = merged_df['measurement_description'].astype(
    str)
merged_df['metric_serving_amount'] = merged_df['metric_serving_amount'].astype(
    float)
merged_df['metric_serving_unit'] = merged_df['metric_serving_unit'].astype(str)
merged_df['monounsaturated_fat'] = merged_df['monounsaturated_fat'].astype(
    float)
merged_df['number_of_units'] = merged_df['number_of_units'].astype(float)
merged_df['polyunsaturated_fat'] = merged_df['polyunsaturated_fat'].astype(
    float)
merged_df['potassium'] = merged_df['potassium'].astype(float)
merged_df['protein'] = merged_df['protein'].astype(float)
merged_df['saturated_fat'] = merged_df['saturated_fat'].astype(float)
merged_df['serving_description'] = merged_df['serving_description'].astype(str)
merged_df['serving_id'] = merged_df['serving_id'].astype(str)
merged_df['serving_url'] = merged_df['serving_url'].astype(str)
merged_df['sodium'] = merged_df['sodium'].astype(float)
merged_df['sugar'] = merged_df['sugar'].astype(float)
merged_df['trans_fat'] = merged_df['trans_fat'].astype(float)
merged_df['vitamin_a'] = merged_df['vitamin_a'].astype(float)
merged_df['vitamin_c'] = merged_df['vitamin_c'].astype(float)
merged_df['added_sugars'] = merged_df['added_sugars'].astype(float)
merged_df['vitamin_d'] = merged_df['vitamin_d'].astype(float)
merged_df['gender'] = merged_df['gender'].astype(str)
merged_df['weight'] = merged_df['weight'].astype(float)
merged_df['id_user'] = merged_df['id_user'].astype(str)
merged_df['activityLevel'] = merged_df['activityLevel'].astype(str)
merged_df['goals'] = merged_df['goals'].astype(str)
merged_df['height'] = merged_df['height'].astype(str)
merged_df['disease'] = merged_df['disease']
merged_df['favoriteFood'] = merged_df['favoriteFood'].astype(str)
merged_df['birthdate'] = merged_df['birthdate'].astype(str)

# Define the feature columns
feature_columns = [
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "food_id", merged_df["food_id"].unique()),
        dimension=32
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "food_name", merged_df["food_name"].unique()),
        dimension=32
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "food_type", merged_df["food_type"].unique()),
        dimension=8
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "brand_name", merged_df["brand_name"].unique()),
        dimension=16
    ),
    tf.feature_column.numeric_column("calcium"),
    tf.feature_column.numeric_column("calories"),
    tf.feature_column.numeric_column("carbohydrate"),
    tf.feature_column.numeric_column("cholesterol"),
    tf.feature_column.numeric_column("fat"),
    tf.feature_column.numeric_column("fiber"),
    tf.feature_column.numeric_column("iron"),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "measurement_description", merged_df["measurement_description"].unique()),
        dimension=8
    ),
    tf.feature_column.numeric_column("metric_serving_amount"),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "metric_serving_unit", merged_df["metric_serving_unit"].unique()),
        dimension=8
    ),
    tf.feature_column.numeric_column("monounsaturated_fat"),
    tf.feature_column.numeric_column("number_of_units"),
    tf.feature_column.numeric_column("polyunsaturated_fat"),
    tf.feature_column.numeric_column("potassium"),
    tf.feature_column.numeric_column("protein"),
    tf.feature_column.numeric_column("saturated_fat"),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "serving_description", merged_df["serving_description"].unique()),
        dimension=8
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "serving_id", merged_df["serving_id"].unique()),
        dimension=32
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "favoriteFood", merged_df["favoriteFood"].unique()),
        dimension=8
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "birthdate", merged_df["birthdate"].unique()),
        dimension=16
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "gender", merged_df["gender"].unique()),
        dimension=8
    ),
    tf.feature_column.numeric_column("weight"),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "goals", merged_df["goals"].unique()),
        dimension=8
    ),
    tf.feature_column.embedding_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            "disease", merged_df["disease"].unique()),
        dimension=8
    )
]

# Create an instance of the retrieval task
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(
        candidates=merged_df["food_id"].unique().tolist()
    )
)




In [ ]:
feature_columns


[EmbeddingColumn(categorical_column=VocabularyListCategoricalColumn(key='food_id', vocabulary_list=('2655309', '58704864', '68848666', '515347', '1684073', '360970', '51780578', '35362184', '1751238', '59433475', '3451226', '1565707', '855710', '6926210', '50672307', '226469', '54876480', '1277715', '2277254', '6721055', '63385', '5405110', '57008676', '2464847', '195215', '1866283', '22836405', '36346', '94228', '48317129', '1905252', '15766420', '406466', '12173623', '3080414', '372433', '9238078', '7556379', '3310602', '152278', '6708', '110855', '3298228', '13582137', '4689586', '41361247', '30316181', '28343875', '19351319', '16694983', '46783291', '54274347', '12100664', '13385806', '6231843', '59526', '568852', '5569712', '1023391', '1824240', '745530', '4055482', '13753575', '71665', '2960716', '170061', '82162', '91509', '13418168', '69128', '1676706', '1155703', '3249398', '247366', '5777', '69083', '361022', '1460350', '4826631', '5873757', '65437539', '17775578', '14718941'

In [ ]:
# Create an instance of the TFRS model and compile it
model = tf.keras.Sequential([
    tf.keras.layers.DenseFeatures(feature_columns),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(32, activation="relu"),
    tf.keras.layers.Dense(16, activation="relu"),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

# Train the model with the feature columns
model.fit(merged_df, epochs=10, verbose=1)